In [1]:
import re
import math
import string
import pickle
import gzip
import numpy as np
import pandas as pd

from numpy.linalg import norm
from nltk import word_tokenize
from gensim.models import Doc2Vec
from gensim.parsing.preprocessing import remove_stopwords

/Users/rytheranderson/opt/anaconda3/envs/NLP/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Notes
### Route Search
* Routes can be searched based on similarity to phrases, while also subsetting for location, quality, type, grade, etc.
* For example, the user could search for 13a sport routes with "sustained crimps" in their area and > 3 RQI.
* This relies on quality (i.e. descriptive) descriptions, which some routes have and some don't.

### Route Profiles
* The doc2vec model can be used to create route profiles based on the route description similarity to keywords.
* For example, similarity to "crimp", "sloper", "jug", etc. can be used to profile the holds of a route.
* Style can also be profiled using keywords like "sustained", "powerful", "thin", etc.
* I have gravitated towards using a word2vec model for this purpose, but the doc2vec model is another (similar) option

In [2]:
# load the data, model, and key for accessing route IDs

model = Doc2Vec.load('doc2vec.model')

with open('docID_2_routeID.pkl', 'rb') as key:
    routeID_key = pickle.load(key)
    
df = pd.read_pickle('Curated_OpenBetaAug2020_RytherAnderson.pkl.zip', compression='zip')

In [5]:
# cleaning function, to clean input strings
def clean_desc(desc):
    
    """
        cleans descriptions for use with doc2vec model
    """
    
    desc = str(desc).lower() # lowercase
    desc = remove_stopwords(desc)
    desc = re.sub(r'\s+', ' ', desc) # multiple spaces converted to single spaces
    desc = re.sub('[0-9]', '', desc) # remove digits
    desc = re.sub(r'(?<=\w)-(?=\w)', ' ', desc) # dash replaced with space
    desc = re.sub(f'[{re.escape(string.punctuation)}]', '', desc) # remove punctuation and special characters
    
    tokens = word_tokenize(desc)
    tokens = [t for t in tokens if len(t) > 1] # remove short tokens

    return tokens

# search function, compares to documents in the provided model
def description_search(model, desc, routeID_key, route_data, topn=3):
    
    """
        model is the doc2vec model, desc is the description
        returns all the data (contained in route_data) for the top N routes
    """
    
    tokens = clean_desc(desc)
    inferred_vector = model.infer_vector(tokens, epochs=1000)
    sims = model.dv.most_similar([inferred_vector], topn=topn)
    
    res = pd.DataFrame()
    route_data.route_ID = route_data.route_ID.astype(int)

    for doc_id, score in sims:  # make sure the routes are in the correct order

        route_id = routeID_key[doc_id]
        route = route_data.query(f'route_ID == {route_id}').copy()
        route['score'] = score
        res = pd.concat([res, route])
    
    return res, [sim for dID, sim in sims], desc

# writes out relevant data from a route search result, for testing
def parse_search_results(res):
    
    """
        res is the output of the description search function
    """
    
    df, sims, desc = res
    N = len(sims)
    
    print('Results for the following document:')
    print()
    print('"' + desc + '"')
    print()
    print(f'{N} routes returned')
    for data, sim in zip([data for i, data in df.iterrows()], sims):
        print('------------------------------------------------------------------------------------------------------------------------')
        print('{:<40} {:<7} {:<5} {:<10}'.format('Name', 'Type', 'Grade', 'ID'))
        grade = ' '.join([g for g in (data['YDS'], data['Vermin']) if g != None])
        print('{:<40} {:<7} {:<5} {:<10}'.format(data['route_name'][0:40], data['type_string'], grade, data['route_ID']))
        print()
        print(f'DESCRIPTION (similarity = {np.round(sim,3)}):')
        
        desc = ' '.join(data['description'])
        NC = len(desc)
        ceil100 = math.ceil(NC/120)
        
        for i in range(ceil100):
            print(desc[i*120:(i+1)*120])
    print('------------------------------------------------------------------------------------------------------------------------')
    

In [19]:
# test some longer phrases

test = [
'Clean vertical face with small crimps',
'Enduro climbing on jugs',
'A long, thin finger crack',
'This route may cause you to void your bowels',
'Just a really massive dyno',
'Lots of pinches',
'Sustained pocket pulling, classic',
'Big dyno to jug for cool people',
]

res = description_search(model, test[-2], routeID_key, df, topn=4)
parse_search_results(res)


Results for the following document:

"Sustained pocket pulling, classic"

4 routes returned
------------------------------------------------------------------------------------------------------------------------
Name                                     Type    Grade ID        
Cowgirl Paradise                         sport   5.11c 105850687 

DESCRIPTION (similarity = 0.735):
Good Pocket pulling.  Steep & sustained.
------------------------------------------------------------------------------------------------------------------------
Name                                     Type    Grade ID        
Ignorant Bliss                           sport   5.11b 113664798 

DESCRIPTION (similarity = 0.733):
Sustained pocket pulling. Excellent as part of the warmup for the other routes at the cliff.
------------------------------------------------------------------------------------------------------------------------
Name                                     Type    Grade ID        
Blue Suede 

In [10]:
search_data = {'route_data': df, 'routeID_key': routeID_key}

with gzip.open('search_data.pkl.zip', 'wb') as out:
    pickle.dump(search_data, out)